In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_test_networkP.csv to train_test_networkP.csv


In [ ]:
import pandas as pd

# Load the dataset
file_name = "train_test_networkP.csv"
df = pd.read_csv(file_name)

# Display dataset shape and columns
print("Dataset Shape:", df.shape)
print("Columns:", df.columns)

Dataset Shape: (211043, 31)
Columns: Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'http_trans_depth', 'http_method',
       'http_version', 'http_request_body_len', 'http_response_body_len',
       'http_status_code', 'Label'],
      dtype='object')


In [ ]:

import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split Labeled (5%) and Unlabeled (95%)
labeled_df = df.sample(frac=0.05, random_state=42)
unlabeled_df = df.drop(labeled_df.index)

# Display the split
print("Labeled Data Shape:", labeled_df.shape)
print("Unlabeled Data Shape:", unlabeled_df.shape)


Labeled Data Shape: (10552, 31)
Unlabeled Data Shape: (200491, 31)


In [ ]:
# from sklearn.model_selection import train_test_split

# label_column = "Label"

# # Drop labels from the Unlabeled Dataset
# unlabeled_df_no_label = unlabeled_df.drop(columns=[label_column])

# # Further split Unlabeled Data (Train 90% / Test 10%)
# unlabeled_train_df, unlabeled_test_df = train_test_split(unlabeled_df_no_label, test_size=0.10, random_state=42)

# # Display updated split sizes
# print("Unlabeled Training Data Shape (without labels):", unlabeled_train_df.shape)
# print("Unlabeled Testing Data Shape (without labels):", unlabeled_test_df.shape)


from sklearn.model_selection import train_test_split

label_column = "Label"

# Save the labels before dropping them
original_labels_unlabeled = unlabeled_df[label_column]  # Store true labels separately

# Drop labels from the Unlabeled Dataset
unlabeled_df_no_label = unlabeled_df.drop(columns=[label_column])

# Further split Unlabeled Data (Train 90% / Test 10%)
unlabeled_train_df, unlabeled_test_df, train_labels, test_labels = train_test_split(
    unlabeled_df_no_label, original_labels_unlabeled, test_size=0.10, random_state=42
)

# Display updated split sizes
print("Unlabeled Training Data Shape (without labels):", unlabeled_train_df.shape)
print("Unlabeled Testing Data Shape (without labels):", unlabeled_test_df.shape)

# Save the true labels for comparison
train_labels.to_csv("unlabeled_train_true_labels.csv", index=False)
test_labels.to_csv("unlabeled_test_true_labels.csv", index=False)
print("True labels saved for comparison later.")



Unlabeled Training Data Shape (without labels): (180441, 30)
Unlabeled Testing Data Shape (without labels): (20050, 30)
True labels saved for comparison later.


In [ ]:
# Save datasets
labeled_df.to_csv("labeled_data.csv", index=False)
print("Labeled dataset saved as 'labeled_data.csv'")

unlabeled_train_df.to_csv("unlabeled_train_data.csv", index=False)
print("Unlabeled train dataset saved as 'unlabeled_train_data.csv'")

unlabeled_test_df.to_csv("unlabeled_test_data.csv", index=False)
print("Unlabeled test dataset saved as 'unlabeled_test_data.csv'")

Labeled dataset saved as 'labeled_data.csv'
Unlabeled train dataset saved as 'unlabeled_train_data.csv'
Unlabeled test dataset saved as 'unlabeled_test_data.csv'


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the labeled dataset
labeled_df = pd.read_csv("labeled_data.csv")

# Define features (X) and labels (y)
label_column = "Label"
X = labeled_df.drop(columns=[label_column])  # Features
y = labeled_df[label_column]  # Labels

# Split into 80% Training and 20% Validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

# Display dataset shapes
print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)


Training Data Shape: (8441, 30)
Validation Data Shape: (2111, 30)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define models
models = {
    "XGBoost": XGBClassifier(eval_metric='logloss'),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Bagging" : BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42)
}

# Train each model and evaluate performance
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_val)  # Predict on validation set
    acc = accuracy_score(y_val, y_pred)  # Calculate accuracy
    print(f"{name} Accuracy: {acc:.4f}")


XGBoost Accuracy: 0.9967
Random Forest Accuracy: 0.9972
Decision Tree Accuracy: 0.9943
Bagging Accuracy: 0.9976
AdaBoost Accuracy: 0.9848


In [ ]:
import pandas as pd

# Load the unlabeled training dataset
unlabeled_train_df = pd.read_csv("unlabeled_train_data.csv")

# Store pseudo-labeled datasets
for name, model in models.items():
    # Generate pseudo labels
    pseudo_labels = model.predict(unlabeled_train_df)

    # Create a new DataFrame with pseudo labels
    pseudo_labeled_df = unlabeled_train_df.copy()
    pseudo_labeled_df["pseudo_label"] = pseudo_labels

    # Save to CSV
    file_name = f"pseudo_labeled_{name.lower().replace(' ', '_')}.csv"
    pseudo_labeled_df.to_csv(file_name, index=False)
    print(f"Pseudo Labels generated using {name} and saved to '{file_name}'")


Pseudo Labels generated using XGBoost and saved to 'pseudo_labeled_xgboost.csv'
Pseudo Labels generated using Random Forest and saved to 'pseudo_labeled_random_forest.csv'
Pseudo Labels generated using Decision Tree and saved to 'pseudo_labeled_decision_tree.csv'
Pseudo Labels generated using Bagging and saved to 'pseudo_labeled_bagging.csv'
Pseudo Labels generated using AdaBoost and saved to 'pseudo_labeled_adaboost.csv'


In [ ]:
import numpy as np
from scipy.stats import mode

# Collect all pseudo-label predictions
predictions = []

for name, model in models.items():
    preds = model.predict(unlabeled_train_df)
    predictions.append(preds)

# Convert predictions list to a NumPy array
predictions = np.array(predictions)

# Apply Majority Voting (mode function selects the most common label)
final_pseudo_labels, _ = mode(predictions, axis=0)

# Create a new DataFrame with final pseudo labels
pseudo_labeled_mv_df = unlabeled_train_df.copy()
pseudo_labeled_mv_df["pseudo_label"] = final_pseudo_labels.flatten()  # Add pseudo labels

# Save the majority-voted pseudo-labeled dataset
pseudo_labeled_mv_df.to_csv("pseudo_labeled_majority_voting.csv", index=False)
print("Pseudo Labels generated using Majority Voting and saved to 'pseudo_labeled_majority_voting.csv'")


Pseudo Labels generated using Majority Voting and saved to 'pseudo_labeled_majority_voting.csv'


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the true labels (ground truth) for the Unlabeled Training Data
true_labels = pd.read_csv("unlabeled_train_true_labels.csv")

# Define model-generated pseudo label file names
pseudo_label_files = {
    "XGBoost": "pseudo_labeled_xgboost.csv",
    "Random Forest": "pseudo_labeled_random_forest.csv",
    "Decision Tree": "pseudo_labeled_decision_tree.csv",
    "Bagging": "pseudo_labeled_bagging.csv",
    "AdaBoost": "pseudo_labeled_adaboost.csv",
    "Majority Voting": "pseudo_labeled_majority_voting.csv"
}

# Store results
accuracy_results = {}

# Iterate through each model’s pseudo-labeled dataset
for model_name, file_name in pseudo_label_files.items():
    # Load pseudo labels
    pseudo_labeled_df = pd.read_csv(file_name)

    # Ensure the number of rows match
    if len(true_labels) != len(pseudo_labeled_df):
        print(f"Warning: Mismatch in row counts for {model_name}. Skipping accuracy calculation.")
        continue

    # Add the true labels to the pseudo-labeled dataset
    comparison_df = pseudo_labeled_df.copy()
    comparison_df["true_label"] = true_labels  # Add original ground truth labels

    # Calculate accuracy
    accuracy = accuracy_score(comparison_df["true_label"], comparison_df["pseudo_label"])
    accuracy_results[model_name] = accuracy
    print(f"{model_name} Pseudo Label Accuracy: {accuracy:.4f}")


XGBoost Pseudo Label Accuracy: 0.9957
Random Forest Pseudo Label Accuracy: 0.9967
Decision Tree Pseudo Label Accuracy: 0.9948
Bagging Pseudo Label Accuracy: 0.9957
AdaBoost Pseudo Label Accuracy: 0.9844
Majority Voting Pseudo Label Accuracy: 0.9965
